# import packages 

In [2]:
import pandas as pd
import os
import dask.dataframe as dd
 


# import OpenAIRE's DOIs
(need a CSV with all OpenAIRES's DOIs, named 'all_dois.csv')

In [3]:
all_dois =  dd.read_csv('all_dois.csv', sep='\t')
all_dois['doi'] = all_dois['doi'].str.lower()
all_dois['doi'] = all_dois['doi'].map_partitions(lambda part: part.str.strip())


# import Semantic Scholar's DOIs
(create a subfolder named 'ss_unique_publications' and save there the Semantic Scholar's Parquet files)

In [4]:
subfolder_path = 'ss_unique_publications'

file_names1 = os.listdir(subfolder_path)
file_names = [file for file in file_names1 if file[0] != '.']

In [5]:
len(file_names)

99

# construct a dask dataframe with Semantic Scholar's data

In [6]:
dois_df = []

for file in file_names:
    df = dd.read_parquet(subfolder_path+'/'+file)
    df = df.dropna(subset=['doi'])
    df = df.reset_index()
    df['doi'] = df['doi'].str.lower()
    df['doi'] = df['doi'].map_partitions(lambda part: part.str.strip())

    df = df[['doi','title']]

    dois_df.append(df)
    
ss_df = dd.concat(dois_df)
ss_df = ss_df.reset_index()

# find which DOIs are in Semantic Scholar but not in OpenAIRE
(it will need some time to run)

In [7]:
common_df = dd.merge(ss_df, all_dois, on = 'doi')

common_dois_set = set(common_df['doi'].compute()) 
not_in_common = ss_df[~ss_df['doi'].isin(common_dois_set)]
not_common_dois_set = set(not_in_common['doi'].compute())
with open("ss_not_open.txt", "w") as file:
    # Iterate through the list
        for item in not_common_dois_set:
            # Write each item to the file
            file.write(item + "\n")
    

In [8]:
len(not_common_dois_set)

4767729